In [9]:

  import pandas as pd
customers_file = 'Customers.csv.xlsx'
products_file = 'Products.csv.xlsx'
customers_df = pd.read_excel(customers_file)
products_df = pd.read_excel(products_file)
customers_df.head(), products_df.head()


(  ï»¿CustomerID        CustomerName         Region SignupDate
 0         C0001    Lawrence Carroll  South America 2022-07-10
 1         C0002      Elizabeth Lutz           Asia 2022-02-13
 2         C0003      Michael Rivera  South America 2024-03-07
 3         C0004  Kathleen Rodriguez  South America 2022-10-09
 4         C0005         Laura Weber           Asia 2022-08-15,
   ï»¿ProductID              ProductName     Category   Price
 0         P001     ActiveWear Biography        Books  169.30
 1         P002    ActiveWear Smartwatch  Electronics  346.30
 2         P003  ComfortLiving Biography        Books   44.12
 3         P004            BookWorld Rug   Home Decor   95.69
 4         P005          TechPro T-Shirt     Clothing  429.31)

In [10]:
customers_df.columns = customers_df.columns.str.strip().str.replace("ï»¿", "")
products_df.columns = products_df.columns.str.strip().str.replace("ï»¿", "")
customers_info = customers_df.info()
products_info = products_df.info()
unique_regions = customers_df['Region'].unique()
unique_categories = products_df['Category'].unique()
customers_info, products_info, unique_regions, unique_categories

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    200 non-null    object        
 1   CustomerName  200 non-null    object        
 2   Region        200 non-null    object        
 3   SignupDate    200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 6.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB


(None,
 None,
 array(['South America', 'Asia', 'North America', 'Europe'], dtype=object),
 array(['Books', 'Electronics', 'Home Decor', 'Clothing'], dtype=object))

In [11]:
import numpy as np
np.random.seed(42)
transactions = []

customer_ids = customers_df['CustomerID'].tolist()
product_ids = products_df['ProductID'].tolist()

for customer_id in customer_ids:
    num_purchases = np.random.randint(5, 16)  # Each customer buys 5-15 products
    purchased_products = np.random.choice(product_ids, num_purchases, replace=False)
    for product_id in purchased_products:
        transactions.append({"CustomerID": customer_id, "ProductID": product_id})
transactions_df = pd.DataFrame(transactions)
transactions_df.head()


,CustomerID,ProductID
0,C0001,P084
1,C0001,P054
2,C0001,P071
3,C0001,P046
4,C0001,P045


In [12]:
transactions_merged = transactions_df.merge(products_df, on="ProductID", how="left")
customer_features = (
    transactions_merged.groupby("CustomerID")
    .agg(
        total_spend=("Price", "sum"),
        product_count=("ProductID", "count"),
        books_count=("Category", lambda x: (x == "Books").sum()),
        electronics_count=("Category", lambda x: (x == "Electronics").sum()),
        home_decor_count=("Category", lambda x: (x == "Home Decor").sum()),
        clothing_count=("Category", lambda x: (x == "Clothing").sum()),
    )
    .reset_index()
)
category_columns = ["books_count", "electronics_count", "home_decor_count", "clothing_count"]
for col in category_columns:
    customer_features[col] = customer_features[col] / customer_features["product_count"]
customer_features.head()


,CustomerID,total_spend,product_count,books_count,electronics_count,home_decor_count,clothing_count
0,C0001,2860.33,11,0.272727,0.000000,0.272727,0.454545
1,C0002,1708.70,7,0.285714,0.000000,0.285714,0.428571
2,C0003,2840.49,9,0.333333,0.333333,0.222222,0.111111
3,C0004,4309.56,15,0.133333,0.266667,0.200000,0.400000
4,C0005,2316.00,7,0.428571,0.142857,0.428571,0.000000


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
features = customer_features.set_index("CustomerID").drop(columns=["total_spend", "product_count"])
similarity_matrix = cosine_similarity(features)
similarity_df = pd.DataFrame(similarity_matrix, index=features.index, columns=features.index)
top_20_customers = customer_features["CustomerID"].head(20)
lookalike_map = {}

for customer_id in top_20_customers:
    similar_customers = (
        similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    )
    lookalike_map[customer_id] = list(similar_customers.items())
lookalike_list = []
for cust_id, lookalikes in lookalike_map.items():
    for similar_cust_id, score in lookalikes:
        lookalike_list.append({"CustomerID": cust_id, "SimilarCustomerID": similar_cust_id, "Score": score})

lookalike_df = pd.DataFrame(lookalike_list)
output_file = "desktop/Rithin_P_Vali_Lookalike.csv"
lookalike_df.to_csv(output_file, index=False)

output_file


'desktop/Rithin_P_Vali_Lookalike.csv'